###### Code accompanies *Natural Language Processing* by KJG Mazidi, all rights reserved.

# Keras Example

Using an SMS Spam data set (slightly modified) from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection). The data set is a collection of 5574 SMS messages that have been labeled as ham or spam. The file is a tab-delimited file with the first column the label and the second the message content. I edited the data set to remove some unwanted columns and add headings. 



In [1]:
# some necessary packages
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras import layers, models

from sklearn.preprocessing import LabelEncoder
import pickle
import numpy as np
import pandas as pd

# set seed for reproducibility
np.random.seed(1234)

In [3]:
df = pd.read_csv('../data/sms-spam.csv', header=0, usecols=[1,2], encoding='latin-1')
print('rows and columns:', df.shape)
print(df.head())

rows and columns: (4837, 2)
   spam                                               text
0     0  Go until jurong point, crazy.. Available only ...
1     0                      Ok lar... Joking wif u oni...
2     1  Free entry in 2 a wkly comp to win FA Cup fina...
3     0  U dun say so early hor... U c already then say...
4     0  Nah I don't think he goes to usf, he lives aro...


In [4]:
# split df into train and test
i = np.random.rand(len(df)) < 0.8
train = df[i]
test = df[~i]
print("train data size: ", train.shape)
print("test data size: ", test.shape)

train data size:  (3856, 2)
test data size:  (981, 2)


In [5]:
# set up X and Y
num_labels = 2
vocab_size = 25000
batch_size = 100

# fit the tokenizer on the training data
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(train.text)

x_train = tokenizer.texts_to_matrix(train.text, mode='tfidf')
x_test = tokenizer.texts_to_matrix(test.text, mode='tfidf')

encoder = LabelEncoder()
encoder.fit(train.spam)
y_train = encoder.transform(train.spam)
y_test = encoder.transform(test.spam)

# check shape
print("train shapes:", x_train.shape, y_train.shape)
print("test shapes:", x_test.shape, y_test.shape)
print("test first five labels:", y_test[:5])


train shapes: (3856, 25000) (3856,)
test shapes: (981, 25000) (981,)
test first five labels: [0 1 1 1 0]


In [6]:
# fit model
model = models.Sequential()
model.add(layers.Dense(32, input_dim=vocab_size, kernel_initializer='normal', activation='relu'))
model.add(layers.Dense(1, kernel_initializer='normal', activation='sigmoid'))
 
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
 
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=30,
                    verbose=1,
                    validation_split=0.1)

Epoch 1/30
35/35 [==============================] - 0s 13ms/step - loss: 0.5738 - accuracy: 0.8046 - val_loss: 0.4167 - val_accuracy: 0.9145
Epoch 2/30
35/35 [==============================] - 0s 8ms/step - loss: 0.3004 - accuracy: 0.9452 - val_loss: 0.2135 - val_accuracy: 0.9715
Epoch 3/30
35/35 [==============================] - 0s 7ms/step - loss: 0.1359 - accuracy: 0.9876 - val_loss: 0.1236 - val_accuracy: 0.9845
Epoch 4/30
35/35 [==============================] - 0s 7ms/step - loss: 0.0683 - accuracy: 0.9954 - val_loss: 0.0920 - val_accuracy: 0.9870
Epoch 5/30
35/35 [==============================] - 0s 8ms/step - loss: 0.0402 - accuracy: 0.9980 - val_loss: 0.0797 - val_accuracy: 0.9870
Epoch 6/30
35/35 [==============================] - 0s 8ms/step - loss: 0.0263 - accuracy: 0.9983 - val_loss: 0.0739 - val_accuracy: 0.9870
Epoch 7/30
35/35 [==============================] - 0s 8ms/step - loss: 0.0183 - accuracy: 0.9997 - val_loss: 0.0719 - val_accuracy: 0.9870
Epoch 8/30
35/35 [=

In [7]:
# evaluate
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('Accuracy: ', score[1])

10/10 [==============================] - 0s 3ms/step - loss: 0.1439 - accuracy: 0.9857
Accuracy:  0.9857288599014282


In [8]:
print(score)

[0.14387717843055725, 0.9857288599014282]


In [9]:
# get predictions so we can calculate more metrics
pred = model.predict(x_test)
pred_labels = [1 if p>0.5 else 0 for p in pred]

In [10]:
pred[:10]

array([[2.5174613e-06],
       [9.9997467e-01],
       [9.9497914e-01],
       [8.6666059e-01],
       [1.3214955e-05],
       [3.9994189e-10],
       [5.6382596e-05],
       [5.2508712e-04],
       [6.1390595e-13],
       [1.6169606e-05]], dtype=float32)

In [11]:
pred_labels[:10]

[0, 1, 1, 1, 0, 0, 0, 0, 0, 0]

In [12]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
print('accuracy score: ', accuracy_score(y_test, pred_labels))
print('precision score: ', precision_score(y_test, pred_labels))
print('recall score: ', recall_score(y_test, pred_labels))
print('f1 score: ', f1_score(y_test, pred_labels))

accuracy score:  0.9857288481141692
precision score:  0.9917355371900827
recall score:  0.9022556390977443
f1 score:  0.9448818897637795


This was by far the most accurate. Much more work could be done modifying the network topology. 